In [1]:
import os
spark_home = os.path.abspath(os.getcwd() + "/../spark-3.5.5-bin-hadoop3")
hadoop_home = os.path.abspath(os.getcwd() + "/../winutils")
print(f"I am using the following SPARK_HOME: {spark_home}")
if os.name == 'nt':
    os.environ["HADOOP_HOME"] = f"{hadoop_home}"
    print(f"Windows detected: set HADOOP_HOME to: {os.environ['HADOOP_HOME']}")
    hadoop_bin = os.path.join(hadoop_home, "bin")
    os.environ["PATH"] = f"{hadoop_bin};{os.environ['PATH']}"
    print(f"  Also added Hadoop bin directory to PATH: {hadoop_bin}")

import findspark
import pyspark
from pyspark.streaming import StreamingContext

findspark.init(spark_home)
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession.builder.getOrCreate()

I am using the following SPARK_HOME: /Users/chenpinyu/Desktop/spark/spark-3.5.5-bin-hadoop3


25/05/04 15:30:40 WARN Utils: Your hostname, Pins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.46.219.58 instead (on interface en0)
25/05/04 15:30:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 15:30:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [3]:
import random
import torch
import joblib
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pyspark.sql import SparkSession, Row

import json

In [4]:
model_path = "./saved_model"
tokenizer_path = "./saved_tokenizer"
label_encoder_path = "./label_encoder.joblib"
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
label_encoder = joblib.load(label_encoder_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2', '3': 'LABEL_3', '4': 'LABEL_4', '5': 'LABEL_5', '6': 'LABEL_6', '7': 'LABEL_7', '8': 'LABEL_8', '9': 'LABEL_9', '10': 'LABEL_10', '11': 'LABEL_11', '12': 'LABEL_12', '13': 'LABEL_13', '14': 'LABEL_14', '15': 'LABEL_15', '16': 'LABEL_16', '17': 'LABEL_17', '18': 'LABEL_18', '19': 'LABEL_19'}. The number of labels will be overwritten to 20.


In [5]:


models_loaded = False
my_model = None
my_tokenizer = None
label_encoder = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Define device


# Initialize StreamingContext
ssc = StreamingContext(sc, 10)


def load_models():
#    print("Loading models...")
    model_path = "./saved_model"
    tokenizer_path = "./saved_tokenizer"
    label_encoder_path = "./label_encoder.joblib"

    loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    loaded_model = AutoModelForSequenceClassification.from_pretrained(model_path).to("mps") # mps

    try:
        loaded_label_encoder = joblib.load(label_encoder_path)
        label_names = loaded_label_encoder.classes_
        print("Label encoder loaded successfully.") # success message
    except Exception as e:
        print(f"Failed to load label encoder: {e}")
        loaded_label_encoder = None
        label_names = None # set to none

    print("Models loaded successfully.")
#    print(f"Loaded model: {loaded_model}")
#    print(f"Label encoder: {loaded_label_encoder}")
    return loaded_model, loaded_tokenizer, loaded_label_encoder

def predict(df):
    global my_model, my_tokenizer, label_encoder
    print("predicting")
    if my_model is None or my_tokenizer is None:
        print("Model and Tokenizer not loaded")
        return None
    
    text_list = [row.text for row in df.collect()]
    inputs = my_tokenizer(text_list, padding=True, truncation=True, return_tensors="pt").to("mps") # mps
    my_model.eval()
    with torch.no_grad():
        outputs = my_model(**inputs)
    predictions = outputs.logits.argmax(dim=-1).cpu().numpy() # argmax
    
    if label_encoder is not None:
        predicted_labels = [label_encoder.classes_[p] for p in predictions]
        return predicted_labels
    else:
        return predictions

def process(time, rdd):
    global models_loaded, my_model, my_tokenizer, label_encoder
    if rdd.isEmpty():
        return

    print("========= %s =========" % str(time))

    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()

    # Load in the model if not yet loaded:
    if not models_loaded:
        # load in your models here
        model, tokenizer, encoder = load_models()
        my_model = model
        my_tokenizer = tokenizer
        label_encoder = encoder
        models_loaded = True

    # Convert RDD of JSON strings to a list of dicts
    records = rdd.map(lambda x: json.loads(x)).collect()

    if not records:
        return
    for r in records:
        if "main_category" in r and r["main_category"]:
            r["main_category"] = r["main_category"].split('.')[0]
        title = r.get("title", "")
        summary = r.get("summary", "")
        r["text"] = title + " " + summary

    # Create a Spark DataFrame for prediction
    prediction_df = spark.createDataFrame([Row(text=rec["text"]) for rec in records if "text" in rec])

    # Make predictions using the predict function
    predicted_categories = predict(prediction_df)

    # Add predictions back to records and print
    if predicted_categories:
        for record, pred_label in zip(records, predicted_categories):
            record["predicted_category"] = pred_label
            #print(record)

        result_rows = [Row(**record) for record in records]
        result_df = spark.createDataFrame(result_rows)
        result_df.select("title", "main_category", "predicted_category").show(truncate=False)
    else:
        print("No predictions made for this batch.")

lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

# Start the streaming context (you can stop later using ssc.stop())
ssc.start()
#ssc.awaitTermination()

/Users/chenpinyu/Desktop/spark/.pixi/envs/default/lib/python3.11/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
25/05/04 15:31:14 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:14 WARN BlockManager: Block input-0-1746365473800 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:31:14 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:14 WARN BlockManager: Block input-0-1746365474000 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:31:14 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:14 WARN BlockManager: Block input-0-1746365474200 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:31:14 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:14 WARN BlockManager: Block input-0-1746365474400 re

========= 2025-05-04 15:31:20 =========


25/05/04 15:31:20 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:20 WARN BlockManager: Block input-0-1746365480600 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:31:22 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:22 WARN BlockManager: Block input-0-1746365482000 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:31:22 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:22 WARN BlockManager: Block input-0-1746365482200 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:31:22 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:22 WARN BlockManager: Block input-0-1746365482400 replicated to only 0 peer(s) instead of 1 peers
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2', '3': 'LABEL_3', '4': 'LABEL_4', '5': 'LABEL_5', '6': 

+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+
|                 aid|          categories|     main_category|           published|             summary|               title|
+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+
|http://arxiv.org/...|             math.FA|           math.FA|2025-04-30T13:32:00Z|While the classic...|The Quantitative ...|
|http://arxiv.org/...|cond-mat.stat-mec...|cond-mat.stat-mech|2025-04-30T13:33:06Z|We investigate th...|Superconductivity...|
|http://arxiv.org/...|quant-ph,cond-mat...|          quant-ph|2025-04-30T13:33:10Z|Non-equilibrium d...|Skin Effect Induc...|
|http://arxiv.org/...|cs.IT,eess.IV,mat...|             cs.IT|2025-04-30T13:34:06Z|To efficiently co...|Fast Sign Retriev...|
|http://arxiv.org/...|     math.ST,stat.TH|           math.ST|2025-04-30T13:34:14Z|Estimating some m...|Convergence ra

25/05/04 15:31:22 WARN BlockManager: Block input-0-1746365482600 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:31:23 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:31:23 WARN BlockManager: Block input-0-1746365482800 replicated to only 0 peer(s) instead of 1 peers
[Stage 0:>                                                          (0 + 1) / 1]

...

25/05/04 15:32:21 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:21 WARN BlockManager: Block input-0-1746365541000 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:21 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:21 WARN BlockManager: Block input-0-1746365541200 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:21 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:21 WARN BlockManager: Block input-0-1746365541400 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:21 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:21 WARN BlockManager: Block input-0-1746365541600 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:23 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:23 WARN BlockManager: Block input-0-1746365543000 replicated to

+--------------------------------------------------------------------------------------------------------------------------+-------------+------------------+
|title                                                                                                                     |main_category|predicted_category|
+--------------------------------------------------------------------------------------------------------------------------+-------------+------------------+
|LLM-based Interactive Imitation Learning for Robotic Manipulation                                                         |cs           |cs                |
|LASHED: LLMs And Static Hardware Analysis for Early Detection of RTL\n  Bugs                                              |cs           |cs                |
|Anatomical Similarity as a New Metric to Evaluate Brain Generative\n  Models                                              |cs           |eess              |
|Solving Copyright Infringement on Short Video Platf

25/05/04 15:32:27 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:27 WARN BlockManager: Block input-0-1746365547000 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:27 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:27 WARN BlockManager: Block input-0-1746365547200 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:27 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:27 WARN BlockManager: Block input-0-1746365547400 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:27 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:27 WARN BlockManager: Block input-0-1746365547600 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:28 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:28 WARN BlockManager: Block input-0-1746365547800 replicated to

========= 2025-05-04 15:32:30 =========
+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|                 aid|          categories|    main_category|           published|             summary|               title|
+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+
|http://arxiv.org/...|   cond-mat.supr-con|cond-mat.supr-con|2025-04-30T16:51:09Z|We investigate th...|Vortex flow aniso...|
|http://arxiv.org/...|stat.ML,cs.LG,sta...|          stat.ML|2025-04-30T16:52:43Z|The Hawkes proces...|Balancing Interpr...|
|http://arxiv.org/...|             math.PR|          math.PR|2025-04-30T16:52:58Z|By taking the vie...|On point interact...|
|http://arxiv.org/...|             math.CO|          math.CO|2025-04-30T16:56:04Z|We prove a conjec...|Girth in $GF(q)$-...|
|http://arxiv.org/...|   cs.SE,cs.AI,cs.CL|            cs.SE|2025-04-30T16:56:06Z|Des

25/05/04 15:32:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:31 WARN BlockManager: Block input-0-1746365551200 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:31 WARN BlockManager: Block input-0-1746365551400 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:31 WARN BlockManager: Block input-0-1746365551600 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:32 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:32 WARN BlockManager: Block input-0-1746365551800 replicated to only 0 peer(s) instead of 1 peers
25/05/04 15:32:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/04 15:32:33 WARN BlockManager: Block input-0-1746365553000 replicated to

In [6]:
ssc.stop()

25/05/04 11:47:04 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
25/05/04 11:47:04 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.base/java.net.SocketInputStream.socketRead0(Native Method)
	at java.base/java.net.SocketInputStream.socketRead(SocketInputStream.java:115)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:168)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:295)
	at java.base/sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:337)
	at java.base/sun.nio.cs.StreamDecoder.read(StreamDecoder.java:179)
	at java.base/java.io.InputStreamReader.read(InputStreamReader.java:181)
	at java.base/java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:326)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:392)
	at org.apache

In [6]:
ssc = StreamingContext(sc, 10)

[Stage 0:>                                                          (0 + 1) / 1]

In [7]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [8]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Users/chenpinyu/Desktop/spark/.pixi/envs/default/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/var/folders/x2/rg0ldb_55rbddbswfqnp3dcw0000gn/T/ipykernel_28353/142111391.py", line 9, in run
  File "/Users/chenpinyu/Desktop/spark/.pixi/envs/default/lib/python3.11/site-packages/pyspark/streaming/context.py", line 226, in start
    self._jssc.start()
  File "/Users/chenpinyu/Desktop/spark/.pixi/envs/default/lib/python3.11/site-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/Users/chenpinyu/Desktop/spark/.pixi/envs/default/lib/python3.11/site-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
           ^^^^^^^^^^^
  File "/Users/chenpinyu/Desktop/spark/.pixi/envs/default/lib/python3.11/site-packages/py4j/protocol.py", line 326, in get_return_value
   

In [9]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


25/05/04 12:29:20 WARN StreamingContext: StreamingContext has not been started yet
